# Clustering, cleaned notebook

### Importing libraries

In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns

from scipy.spatial.distance import pdist, squareform

# import dataset
df_original_copy = pd.read_csv("dataset (missing + split)/train.csv", skipinitialspace=True)
df = pd.read_csv("dataset (missing + split)/train.csv", skipinitialspace=True) #this will be modified

In [3]:
#scaling, normalization
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#kmeans, dbscan, hierarchical (sklearn)
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
#evaluation
from sklearn.metrics import silhouette_score
#import dataset
from sklearn.datasets import load_iris

#distance matrix (dbscan elbow, hierarchical)
from scipy.spatial.distance import pdist, squareform
# hierarchical (scipy)
from scipy.cluster.hierarchy import linkage, dendrogram

### Data transformation

In [4]:
#Changing udm to duration_ms and features_duration_ms from ms to min
df['duration_ms'] *= 1/6e4
#Setting popularity as a %
df['popularity'] /= 100
df.rename(columns = {'duration_ms':'duration_min'}, inplace = True)
df.rename(columns = {'popularity':'popularity_percent'}, inplace = True)

In [5]:
#Creating genre map -> mapping every genre into an int value in order to have correlation values. 
#Redoing Pearson correlation adding genre category
df['genre'].value_counts()
genre_map={"j-dance":0,"iranian":1,"brazil":2,"chicago-house":3,"forro":4,"idm":5,"indian":6,"study":7,"disney":8,"afrobeat":9,"mandopop":10,"techno":11,"sleep":12,"spanish":13,"j-idol":14,"industrial":15,"happy":16,"bluegrass":17,"black-metal":18,"breakbeat":19}
df.replace({'genre':genre_map}).corr(numeric_only=True).style.background_gradient(cmap='coolwarm',vmin=-1,vmax=1) #!!!

,duration_min,explicit,popularity_percent,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,features_duration_ms,time_signature,n_beats,n_bars,popularity_confidence,processing,genre
duration_min,1.000000,-0.048723,-0.076202,-0.080932,0.102996,0.016712,0.039403,-0.023310,-0.074774,-0.184902,0.079685,-0.021184,-0.141938,0.048765,0.999918,0.009033,0.839313,0.838041,-0.005830,-0.010400,0.030701
explicit,-0.048723,1.000000,-0.013096,0.056992,0.124405,-0.000991,0.131777,-0.042846,0.294287,-0.113975,-0.125101,0.006995,0.008631,0.016972,-0.048699,0.051260,-0.039830,-0.039435,0.029119,-0.005878,-0.031463
popularity_percent,-0.076202,-0.013096,1.000000,0.051046,-0.056547,-0.008992,0.046703,0.073043,-0.096382,0.078424,-0.266843,0.027105,0.077652,-0.024162,-0.076228,-0.003160,-0.084774,-0.089134,-0.003665,0.008733,0.035557
danceability,-0.080932,0.056992,0.051046,1.000000,0.193940,0.035311,0.385244,-0.072422,0.111454,-0.199082,-0.202316,-0.174193,0.559578,0.092267,-0.080891,0.292167,-0.033604,-0.069332,0.011602,-0.026273,-0.125284
energy,0.102996,0.124405,-0.056547,0.193940,1.000000,0.057815,0.720907,-0.079969,0.143454,-0.699454,-0.203111,0.191024,0.284069,0.331538,0.102985,0.204646,0.243586,0.208615,0.040022,-0.037847,0.236734
key,0.016712,-0.000991,-0.008992,0.035311,0.057815,1.000000,0.047718,-0.157515,0.018157,-0.049867,-0.001532,-0.007420,0.028221,0.022414,0.016786,0.028388,0.027133,0.022042,0.019790,-0.490366,0.018896
loudness,0.039403,0.131777,0.046703,0.385244,0.720907,0.047718,1.000000,-0.033544,0.109793,-0.552519,-0.447106,0.047502,0.377743,0.308203,0.039415,0.250568,0.165565,0.131064,0.049174,-0.020563,0.119055
mode,-0.023310,-0.042846,0.073043,-0.072422,-0.079969,-0.157515,-0.033544,1.000000,-0.064289,0.085246,-0.080035,0.015738,-0.006907,-0.013001,-0.023409,-0.019378,-0.030685,-0.027687,0.003698,0.261129,0.056051
speechiness,-0.074774,0.294287,-0.096382,0.111454,0.143454,0.018157,0.109793,-0.064289,1.000000,-0.087997,-0.115803,0.050034,0.056068,0.062161,-0.074722,0.080739,-0.043217,-0.040820,0.026399,-0.022909,-0.118012
acousticness,-0.184902,-0.113975,0.078424,-0.199082,-0.699454,-0.049867,-0.552519,0.085246,-0.087997,1.000000,0.094256,-0.075924,-0.131961,-0.244244,-0.184846,-0.139706,-0.283324,-0.248164,-0.021155,0.022684,-0.196470


## Clustering searching for genre division 
I want to cluster considering only 'musical features', that are the 'technical' ones. The idea is to find the already-given division by genre.  
It is interesting, in case this works, to do a **hierarchical clustering** in order to search for 'base' genres and subgenres. 

The interesting features for the scope could be:
- `duration_min`
- `key` (discrete); 
- `mode` (discrete);
- `speechiness`;
- `acousticness`;
- `instrumentalness`;
- `tempo`;
- `time_signature`;
- `n_beats`;
- `n_bars`;  
being the latter two highly correlated with `duration_ms`, just the former is necessary. 
also `genre` in order to check the results.  
Being those the interesting attributes, a goal-specific data understanding will be done only on this data. 

Because K-means algorithm works only with continuous attributes, also a data subset with the following attributes is created and used:
- `duration_min`
- `popularity`;
- `danceability`;
- `energy`; 
- `loudness`; (trying to eliminate bc of strong correlation to `energy`)
- `speechiness`;
- `acousticness`;
- `instrumentalness`;
- `liveness`;
- `valence`;
- `tempo`.

### Outlier criterion
The outlier criterion for the following scope is composed by the following cut:
- `time_sigmature` > 1.0, in order to select only _music tracks_, where the features taken into spot are all well-defined;
- the first criterion ideally is enough but `sleep` tracks containing noise sounds often are bad-detected by the algorithm. Bc of that, in this first phase the `sleep` tracks are all excluded.  

The dataset will be tried at first without outliers [], after that, outliers will be reintroduced and clustering redone [].

### Filling `NaN` values

In [6]:
#Dealing with mode attribute missing values
#Computing p0 and p1 and filling missing values of mode attribute by sampling
#df['mode'].value_counts().plot(kind='bar', title='mode')
p0=df['mode'].value_counts()[0]/(len(df)-df['mode'].isnull().sum())
p1=df['mode'].value_counts()[1]/(len(df)-df['mode'].isnull().sum())
#print(f'p0={p0} , p1={p1}')
list_of_nan_indexes=df[df['mode'].isnull()].index.tolist()
for i in list_of_nan_indexes:
    #df.loc[i,'mode']
    #df.loc[['viper', 'sidewinder'], ['shield']] = 50
    if np.random.random() < p1:
        df.loc[i,'mode'] = 1.0
    else:
        df.loc[i,'mode'] = 0.0

In [7]:
#Dealing with time_signature attribute missing values
#Computing the array containing the probabilities of every outcome for time_signature
outcomes_of_time_signature = len(df['time_signature'].value_counts())
p_array=np.array(df['time_signature'].value_counts().sort_index(ascending=True)/(len(df)-df['time_signature'].isnull().sum()))
print(p_array)
#creating a dictionary containing the correspondance between value and sorted index
dict_ts = {0: 0.0, 1: 1.0, 2: 3.0, 3 : 4.0, 4 : 5.0}
list_of_nan_indexes_ts=df[df['time_signature'].isnull()].index.tolist()
#print(list_of_nan_indexes)
print(len(list_of_nan_indexes_ts))
from scipy.stats import multinomial 
#please forgive the dunb programming here, no real deal of time, only ~3000 points to be evaluated
for i in list_of_nan_indexes_ts:
    tmp = multinomial.rvs(1, p_array, size=1, random_state=None)
    array_tmp=np.where(tmp[0][:]==1)
    index=array_tmp[0][0] #implement a dict for the substitution
    df.loc[i,'time_signature'] = dict_ts[index]

[0.00711084 0.01151646 0.08981295 0.86257536 0.02898439]
2062


In [8]:
#Deleting outliers
df = df[(df['time_signature'] >1) & (df['genre'].str.contains('sleep')==False)]
df

,name,duration_min,explicit,popularity_percent,artists,album_name,danceability,energy,key,loudness,...,liveness,valence,tempo,features_duration_ms,time_signature,n_beats,n_bars,popularity_confidence,processing,genre
0,Long Road,4.029333,False,0.46,Funki Porcini,Hed Phone Sex,0.690,0.513,5,-12.529,...,0.2810,0.6350,142.221,241760,4.0,567.0,141.0,NaN,1.279305,j-dance
1,"Daniâl My Son, Where Did You Vanish?",7.400000,False,0.00,Siavash Amini,A Trail of Laughters,0.069,0.196,1,-24.801,...,0.3600,0.0336,75.596,444000,4.0,617.0,155.0,NaN,2.367412,iranian
2,Ondskapens Galakse,5.558433,False,0.03,Kvelertak,Nattesferd,0.363,0.854,2,-7.445,...,0.2620,0.4890,116.628,333507,4.0,652.0,162.0,NaN,3.700483,black-metal
3,Can't Look Away,4.496667,False,0.23,The Wood Brothers,One Drop of Truth,0.523,0.585,5,-5.607,...,0.1170,0.3850,140.880,269800,4.0,619.0,155.0,0.469,1.279305,bluegrass
4,Thunderground,5.127517,False,0.25,The Darkraver;DJ Vince,Happy Hardcore Top 100,0.643,0.687,7,-11.340,...,0.2990,0.6370,143.956,307652,4.0,733.0,183.0,NaN,4.067086,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,アンダスタンディング,3.750700,False,0.16,PASSEPIED,ニュイ,0.570,0.875,1,-5.065,...,0.3390,0.8020,175.017,225043,4.0,641.0,159.0,NaN,2.367412,j-idol
14995,Outpt,7.200433,False,0.00,Autechre,EP7,0.554,0.657,1,-14.232,...,0.1320,0.0335,117.319,432027,4.0,837.0,212.0,NaN,2.367412,idm
14997,Psychological Attack,6.668183,False,0.43,Droplex,Psychological Attack,0.799,0.535,1,-9.103,...,0.0713,0.0708,126.000,400091,4.0,837.0,209.0,NaN,2.367412,techno
14998,Don't Hate The 808 (feat. Lafa Taylor),3.287500,False,0.37,Bassnectar;Lafa Taylor,Noise vs Beauty,0.511,0.970,5,-2.299,...,0.4020,0.2700,160.044,197250,4.0,513.0,130.0,NaN,1.279305,breakbeat


In [9]:
#Selecting only the specific columns for the dataset
df_cont_plus_cat=df[['duration_min','key', 'mode','danceability','energy','speechiness','acousticness','instrumentalness','tempo','time_signature']]
df_cont=df[['duration_min','popularity_percent', 'danceability','energy','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

### Measure definition
Being some of those attributes _categorical_, such as `time_signature`, it is necessary to define a custom distance, that encorporates the Euclidean distance for continuous attributes and the dissimilarity concept of distance for categorical attributes.

In [11]:
#Reindexing columns just for defining in an easier manner the custom metric
column_names_cont_and_cat = ['key','duration_min', 'danceability','energy','speechiness', 'acousticness',
       'instrumentalness','tempo','mode','time_signature']


df_cont_plus_cat = df_cont_plus_cat.reindex(columns=column_names_cont_and_cat)
df_cont_plus_cat.columns

Index(['key', 'duration_min', 'danceability', 'energy', 'speechiness',
       'acousticness', 'instrumentalness', 'tempo', 'mode', 'time_signature'],
      dtype='object')

In [15]:
from scipy.spatial.distance import seuclidean, jaccard
# Defining the metric
# key has a custom metric due to its circular shape 
# When the attribute is continuous, then the euclidean measure is used,
# when the attribute is categorical, dissimilarity is computed. 

def custom_dist_key(x1, x2):
    index_key = 0
    index_con = 8
    
    c = 0.09090909
    diff = abs(x1[index_key] - x2[index_key]) / c
    diff_key = (min(12 - diff, diff) * c)
    
    print("x1 key", x1[index_key])
    print("x2 key", x2[index_key])
    print("diff key", diff_key)
    
    d_con = seuclidean(x1[index_key+1:index_con], x2[index_key+1:index_con], V=np.ones(index_con - 1))
    d_cat = jaccard(x1[index_con:], x2[index_con:])
    
    print("x1 con", x1[index_key+1:index_con])
    print("x2 con", x2[index_key+1:index_con])
    print("d_con", d_con)
    
    print("x1 cat", x1[index_con:])
    print("x2 cat", x2[index_con:])
    print("d_cat", d_cat)
    
    return diff_key + d_con + d_cat

In [28]:
indexes=np.array([0,8,9])
arr=np.take(X[::],indexes)
print(arr)

[5. 1. 4.]


In [33]:
# Defining the metric
# When the attribute is continuous, then the euclidean measure is used,
# when the attribute is categorical, dissimilarity is computed. key is categorical 

def custom_dist_cat_and_cont(x1, x2):
    index_key = 0
    index_con = 8
    indexes=np.array([0,8,9])
    cat1=np.take(x1,indexes)
    cat2=np.take(x2,indexes)
    
    d_con = seuclidean(x1[index_key+1:index_con], x2[index_key+1:index_con], V=np.ones(index_con - 1))
    
    d_cat = jaccard(cat1, cat2)
    
    print("x1 con", x1[index_key+1:index_con])
    print("x2 con", x2[index_key+1:index_con])
    print("d_con", d_con)
    
    print("x1 cat", cat1)
    print("x2 cat", cat2)
    print("d_cat", d_cat)
    
    return d_con + d_cat

In [34]:
#Trial done and erased, will be redone after normalization
print(custom_dist_cat_and_cont(scaledX[0], scaledX[4]))

x1 con [0.08124597 0.68607924 0.51299016 0.08153242 0.30321285 0.53653654
 0.58831354]
x2 con [1.04937681e-01 6.35202425e-01 6.86993677e-01 3.39445536e-02
 5.78313253e-04 9.69969970e-02 5.97435372e-01]
d_con 0.566177934382277
x1 cat [0.45454545 1.         0.5       ]
x2 cat [0.63636364 1.         0.5       ]
d_cat 0.3333333333333333
0.8995112677156103


### Normalization (or, standardization)
It is necessary to choose a type of normalization. Looking at the distributions of the attributes for the scope, it is possible to decide with some criteria, see: https://developers.google.com/machine-learning/data-prep/transform/normalization?hl=en .

In [35]:
X_cont_and_cat = df_cont_plus_cat.values #key is included
X_cont = df_cont.values #only continuous values
y = np.array(df['genre'])
y_mapped = pd.DataFrame(y).iloc[:,0]

In [39]:
#performing minmax normalization 
scaler = MinMaxScaler()
X_minmax_cont_and_cat = scaler.fit_transform(X_cont_and_cat)
X_minmax_cont = scaler.fit_transform(X_cont)
#Now we have the two dataset and the metrics!

## Internal measures for cluster validation
In addition to the SSE and Sil coefficient, a metric validated by the following article: https://pubmed.ncbi.nlm.nih.gov/23193245/
has been used, implemented by the authors of a linked repository (see below).
### CVNN index implementation
All right deserved to the authors of the following GitHub repository: https://github.com/clslabMSU/clustGUI/tree/master 

In [40]:
# all rights to the following repository authors: https://github.com/clslabMSU/clustGUI/tree/master 
"""
Created on Sun Apr 23 16:15:37 2017

"""
import numpy as np

import scipy as sy
from math import pow 

class internalIndex:
    def  __init__(self, num_k):
        self.num_k = num_k
        self.class_iter = range(1, num_k +1)
    
    def euclidean_centroid(self, data,label, label_num = False):
        if label_num == False:
            num_attr = data.shape[1]
            centroid = np.zeros([1, num_attr])
            for attr_id in range(num_attr):
                sum_attr_id = 0
                for i in range(len(data)):
                    sum_attr_id += data[i][attr_id]
                centroid[0][attr_id] = sum_attr_id/len(data)
            return centroid[0]
        else:
            count = 0
            for l in label:
                if l == label_num:
                    count +=1
            length = count
            num_attr = data.shape[1]
            centroid = np.zeros([1, num_attr])
            for attr_id in range(num_attr):
                sum_attr_id = 0
                for i in range(len(data)):
                    if label[i] == label_num:
                        sum_attr_id += data[i][attr_id]
                centroid[0][attr_id] = sum_attr_id/length
            return centroid[0]
    def centroid_list(self, data,label):
        c_list = []
        for index_i in self.class_iter:
            c_list.append(self.euclidean_centroid(data,label,index_i))
        return c_list
        
    def element_of_clustert (self, data, label, cluster_i):
        eoc = []
        for l in range(len(data)):
            if label[l] == cluster_i:
                eoc.append(data[l])
        return np.asarray(eoc)
    
    def distance_from_cluster (self, data, label,  cluster_i, centroid_i):
        eoc = self.element_of_clustert(data, label, cluster_i)
        centroid_i = self.euclidean_centroid(data, label, centroid_i)
        distance = 0
        for i in eoc:
            distance += sy.spatial.distance.euclidean(centroid_i, i)
        return distance
    
    def distance_from_cluster_sqr (self, data, label,  cluster_i, centroid_i):
        eoc = self.element_of_clustert(data, label, cluster_i)
        centroid_i = self.euclidean_centroid(data, label, centroid_i)
        distance = 0
        for i in eoc:
            distance += sy.spatial.distance.sqeuclidean(centroid_i, i)
        return distance
    
    
    def nn_exclude(self, data, label, data_i ,label_i ,k):
        data_i = [data_i]
        dist_mat = sy.spatial.distance.cdist(data_i, data)[0]
        dist_mat = np.argsort(dist_mat)[1:k+1]
        count = 0
        for i in dist_mat:
            if label_i != i:
                count = count + 1
        return count

    
    def CVNN(self, data, label ): #data is the array X_minmax, label are the cluster labels
        #Need COM and SEP
        k = 10
        COM = 0
        SEP = []
        for index_i in self.class_iter:
            temp_sep = 0
            eoc  = self.element_of_clustert(data, label, index_i)
            n= len(eoc)
            if n !=1:
                COM = COM + (sum(sy.spatial.distance.pdist(eoc,'euclidean')) * 2 / (n*(n-1)))
            for e in eoc:
                q_index_e = self.nn_exclude(data, label, e, index_i, k )
                temp_sep = temp_sep +(q_index_e / k)
            SEP.append(temp_sep)
        SEP = max(SEP)
        return COM , SEP
    #SEP and COM has to be computed for all NCs in order to construct the
    def CVNN_n(self, COM, SEP): 
        index = self.num_k - 2
        return COM[index] / max(COM) + SEP[index] / max(SEP) 
        
        